In [ ]:
import os, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlrd
from sklearn.preprocessing import normalize, StandardScaler, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from nilearn.connectome import ConnectivityMeasure
from pandas import DataFrame
import scipy as sc
from scipy import io
from scipy.stats import pearsonr
from os.path import join, exists, dirname
from glob import glob
from brainspace import gradient
import nibabel as nib

# Load Parameter

In [ ]:
# demo = pd.read_excel(join(path_data, 'abide_func_fn_abideII_sorted.xlsx'), sheet_name='Sheet1', skiprows=0)
demo = pd.read_excel(join(path_data, 'abide_func_fn_abideII_sorted.xlsx'), sheet_name='male', skiprows=0)


sub_id = demo['ID']
sub_list = demo['FILE_ID']
label2 = demo['Group']
label = demo['DX_GROUP']
site_id = demo['Site']
site_label = demo['SITE_Label']
Age = demo['Age']
FD = demo['MeanFD_Jenkinson'] # = func_mean_fd
FIQ = demo['FIQ']
sex = demo['Sex']

male_index = np.where(sex=='M')[0]

ASD_index = np.where(label == 1)[0]                
TD_index = np.where(label == 2)[0]
Total_index = np.concatenate((ASD_index,TD_index)) 
sorted_idx = np.concatenate((ASD_index,TD_index), axis = 0)

site_label_sorted = np.array(site_label)[sorted_idx]

print('Total : ', len(label))
print('ASD : ', len(ASD_index))
print('TD : ', len(TD_index))

# Compare parameter

In [ ]:
import statsmodels as sm
from statsmodels.stats.multitest import multipletests

i = Age # Age

print('Mean : ', np.array(i)[ASD_index].mean(), 'Std : ', np.array(i)[ASD_index].std())
print('Mean : ', np.array(i)[TD_index].mean(), 'Std : ', np.array(i)[TD_index].std())

[s,p] = sc.stats.ttest_ind(np.array(i)[ASD_index],np.array(i)[TD_index], equal_var=False, axis=0) 

p_fdr = sm.stats.multitest.multipletests(p,alpha=0.05,method='fdr_bh')

print('')
print('t : ',s)
print("p-value")
print('0 : ', np.where(p_fdr[0]==True),'\n', np.where(p_fdr[0]==True)[0].shape)
print(p_fdr)

In [ ]:
# Compare Age among sites

i = Age # Age

label_index = TD_index # ASD_index TD_index Total_index

index_1 = np.intersect1d(np.where(np.array(site_id)=='TCD')[0],ASD_index) # ASD_index TD_index Total_index label_index
index_2 = np.intersect1d(np.where(np.array(site_id)=='NYU')[0],ASD_index)
index_3 = np.intersect1d(np.where(np.array(site_id)=='IP')[0],ASD_index)

index_4 = np.intersect1d(np.where(np.array(site_id)=='TCD')[0],TD_index) # ASD_index TD_index Total_index label_index
index_5 = np.intersect1d(np.where(np.array(site_id)=='NYU')[0],TD_index)
index_6 = np.intersect1d(np.where(np.array(site_id)=='IP')[0],TD_index)

print('Mean : ', np.array(i)[index_1].mean(), 'Std : ', np.array(i)[index_1].std())
print('Mean : ', np.array(i)[index_2].mean(), 'Std : ', np.array(i)[index_2].std())
print('Mean : ', np.array(i)[index_3].mean(), 'Std : ', np.array(i)[index_3].std())

print('Mean : ', np.array(i)[index_4].mean(), 'Std : ', np.array(i)[index_4].std())
print('Mean : ', np.array(i)[index_5].mean(), 'Std : ', np.array(i)[index_5].std())
print('Mean : ', np.array(i)[index_6].mean(), 'Std : ', np.array(i)[index_6].std())

print('TCD : ', len(index_1), ' ', len(index_4))
print('NYU : ', len(index_2), ' ', len(index_5))
print('IP : ', len(index_3), ' ', len(index_6))

[s1,p1] = sc.stats.ttest_ind(np.array(i)[index_1],np.array(i)[index_4], equal_var=False, axis=0) 
[s2,p2] = sc.stats.ttest_ind(np.array(i)[index_2],np.array(i)[index_5], equal_var=False, axis=0) 
[s3,p3] = sc.stats.ttest_ind(np.array(i)[index_3],np.array(i)[index_6], equal_var=False, axis=0) 


p1_fdr = sm.stats.multitest.multipletests(p1,alpha=0.05,method='fdr_bh')
p2_fdr = sm.stats.multitest.multipletests(p2,alpha=0.05,method='fdr_bh')
p3_fdr = sm.stats.multitest.multipletests(p3,alpha=0.05,method='fdr_bh')

print('')
print("p-value")
print('p1 : ', np.where(p1_fdr[0]==True),'\n', np.where(p1_fdr[0]==True)[0].shape)
print(p1_fdr)

print('p2 : ', np.where(p2_fdr[0]==True),'\n', np.where(p2_fdr[0]==True)[0].shape)
print(p2_fdr)

print('p3 : ', np.where(p3_fdr[0]==True),'\n', np.where(p3_fdr[0]==True)[0].shape)
print(p3_fdr)

In [ ]:
ADOS_Total = np.nan_to_num(np.array(demo['ADOS_G_TOTAL']), nan = -1e-16) # -9999 -> -1, nan -> -2
ADOS_Total = np.where(ADOS_Total == -999, -1e-16, ADOS_Total)
ADOS_Total

ADOS_comm = np.nan_to_num(np.array(demo['ADOS_G_COMM']), nan = -1e-16)
ADOS_comm = np.where(ADOS_comm == -999, -1e-16, ADOS_comm)

ADOS_social =  np.nan_to_num(np.array(demo['ADOS_G_SOCIAL']), nan = -1e-16)
ADOS_social = np.where(ADOS_social == -999, -1e-16, ADOS_social)

ADOS_behav =  np.nan_to_num(np.array(demo['ADOS_G_STEREO_BEHAV']), nan = -1e-16)
ADOS_behav = np.where(ADOS_behav == -999, -1e-16, ADOS_behav)

for i in [ADOS_Total, ADOS_comm, ADOS_social, ADOS_behav]:

    print('Total Mean : ', np.array(i)[ASD_index].mean(), 'Std : ', np.array(i)[ASD_index].std()) # ASD_index index_1 index_2 index_3
    print('TCD Mean : ', np.array(i)[index_1].mean(), 'Std : ', np.array(i)[index_1].std()) # ASD_index index_1 index_2 index_3
    print('NYU Mean : ', np.array(i)[index_2].mean(), 'Std : ', np.array(i)[index_2].std()) # ASD_index index_1 index_2 index_3
    print('IP Mean : ', np.array(i)[index_3].mean(), 'Std : ', np.array(i)[index_3].std()) # ASD_index index_1 index_2 index_3
    print('')

# Make functional gradients

In [ ]:
# Make surface based pearson connectivity matrtix
# ABIDE 2

save_ts = False

file_list = glob(join(path_ts,'*npy'))

demo = pd.read_excel(join(path_ts_demo, 'abide_func_fn_abideII.xlsx'), sheet_name='abide_func_fn_abideII', skiprows=0)

ID = demo['ID']
# OrgID = demo['OrgID']
Site = demo['Site']

pcor = ConnectivityMeasure(kind ='correlation') # partial correlation correlation

for i, filename in enumerate(file_list):
    print(i)
    Site_num = np.array(Site)[i]
    ID_num = np.array(ID)[i]
    folder_name = str(Site_num) + '_' + str(ID_num)
#     if not(os.path.isdir(join(path_work,f'{folder_name}'))):
#         print(f'Make folder {folder_name}')
#         os.makedirs(os.path.join(path_work,f'{folder_name}'))
#     if os.path.isdir(join(path_work,f'{folder_name}')):
#         print('pass')
#     else:
#         os.makedirs(os.path.join(path_work,f'{folder_name}'))

    save_path = glob(join(path_work,f'*{ID_num}'))
    print(ID_num , ' ', Site_num, ' ', save_path)
    
    ts = np.load(filename)
    n_roi = len(ts)
    
    if save_ts:
        print(ts.shape)
        np.save(join(path_work, f'{folder_name}/ts_MMP.npy'), ts)
        print('save ts')

    conn_mat = pcor.fit_transform(ts.T.reshape(1,-1,360))[0]
    conn_mat = np.where(np.eye(n_roi) ==1, 0, conn_mat)
#     print(conn_mat)
    
#     np.save(join(save_path[0], 'surf_conn_mat.npy'), conn_mat)

In [ ]:
# Combat harmonization correlation matrix

from nilearn.connectome import ConnectivityMeasure, sym_matrix_to_vec, vec_to_sym_matrix
import neuroCombat

file_list = [join(path_data,f'{workfolder}',f'{sub_list[i]}','surf_conn_mat.npy') for i in range(len(sub_list))]
list_aff = [np.load(sl) for sl in file_list]

n = len(list_aff)

list_aff_vec = [None] * n

for i, x1 in enumerate(list_aff):
    print(i,' ', end = '', flush = True)
    x1 = x1 + np.eye(len(x1))
    list_aff_vec[i] = sym_matrix_to_vec(x1)

site_sorted = site_id[sorted_idx]
siteID = np.array([list(site_sorted).index(x) for x in site_sorted])

covars = pd.DataFrame({'Site' : siteID})
combat_res = neuroCombat.neuroCombat(dat = np.array(list_aff_vec).T, covars = covars, batch_col = 'Site', ref_batch = None) # list(site_sorted).index(ref_site))

surf_conn_mat_har = [None] * n

for i, x1 in enumerate(combat_res.T):
    print(i,' ', end = '', flush = True)
    
    x2 = vec_to_sym_matrix(x1)
    x2 = np.where(np.eye(len(x2)) ==1, 0, x2)
    
    surf_conn_mat_har[i] = x2

for i, filename in enumerate(file_list):
    print(i)
    FileID_num = np.array(sub_list)[i]
    save_path = glob(join(path_work,f'*{FileID_num}'))
    print(FileID_num, ' ', save_path)
      
#     np.save(join(save_path[0], 'surf_conn_mat_Mean_har_male.npy'), np.array(surf_conn_mat_har[i]))

In [ ]:
# FC thresholding

file_list = [join(path_data,f'{workfolder}',f'{sub_list[i]}','surf_conn_mat_Mean_har.npy') for i in range(len(sub_list))]
path_work = join(path_data,f'{workfolder}')

for thresh in [0.99, 0.97, 0.95, 0.9, 0.75, 0.5]:

    sparsity = thresh

    k = str(int(100-sparsity*100))
    k = k.zfill(2)

    print(f'Top {k}')

    for i, x in enumerate(file_list):
        print(f'{i} / {len(file_list)}')
        conn_mat = np.load(x)
        z_conn_mat = np.arctanh(np.nan_to_num(conn_mat, nan=0.0))
        noaff_conn_mat = gradient.compute_affinity(z_conn_mat, sparsity=sparsity)
        aff_conn_mat = gradient.compute_affinity(z_conn_mat, kernel = 'cosine', sparsity = sparsity)
        
        print(f'noaff isnan : {np.isnan(noaff_conn_mat).sum()}, symmetric : {gradient.is_symmetric(noaff_conn_mat)}')
        print(f'aff isnan : {np.isnan(aff_conn_mat).sum()}, symmetric : {gradient.is_symmetric(aff_conn_mat)}')
        
#         np.save(join(path_work, x.split('\\')[-2], f'noaff_surf_mat_Mean_har_top{k}.npy'),noaff_conn_mat)
#         np.save(join(path_work, x.split('\\')[-2], f'aff_surf_mat_Mean_har_top{k}.npy'), aff_conn_mat)

    # grpmean
    subj_num = len(file_list)

    list_aff = [np.load(sl) for sl in file_list]
    grpmean_conn_mat = np.nan_to_num(np.arctanh(np.nan_to_num(np.mean(list_aff, axis = 0), nan=0.0)), nan = 0.0)

    noaff_grpmean_conn_mat = gradient.compute_affinity(grpmean_conn_mat, sparsity=sparsity) # sparsity로 thresholding ratio 조절
    noaff_grpmean_conn_mat = np.nan_to_num(noaff_grpmean_conn_mat,nan=0.0)
    aff_grpmean_conn_mat = gradient.compute_affinity(grpmean_conn_mat, kernel = 'cosine', sparsity=sparsity)
    aff_grpmean_conn_mat = np.nan_to_num(aff_grpmean_conn_mat,nan=0.0)

    # np.save(join(path_work, f'grp_mean.noaff_surf_mat_Mean_har_n{subj_num}_top{k}.npy'), noaff_grpmean_conn_mat)
    # np.save(join(path_work, f'grp_mean.aff_surf_mat_Mean_har_n{subj_num}_top{k}.npy'), aff_grpmean_conn_mat)


In [ ]:
# Dimension reduction
# PCA alignment, DiffuisonMap alignment

from sklearn.decomposition import PCA

embedding = 'PCA' #  'PCA', 'Diffusion'

comp_num = 10

if embedding == 'Diffusion':
    file_list = []
    for i in range(len(sub_list)):
        file_list.append(glob(join(path_data,f'{workfolder}',f'{sub_list[i]}','aff*'))) 
    file_list = np.array(file_list).reshape(-1)
    print(len(file_list))
    list_aff = [np.load(sl) for sl in file_list]
    grp_aff = np.load(join(path_work, f'grp_mean.aff_mat_n{len(file_list)}.npy')) 
    emb = gradient.embedding.DiffusionMaps(n_components = comp_num) 
    print(emb)

elif embedding == 'PCA':
    file_list = []
    for i in range(len(sub_list)):
        file_list.append(glob(join(path_data,f'{workfolder}',f'{sub_list[i]}','noaff_surf_mat_Mean_har_top50.npy'))) 
    file_list = np.array(file_list).reshape(-1)
    print(len(file_list))
    list_aff = [np.load(sl) for sl in file_list] 
    grp_aff = np.load(join(path_work,  f'grp_mean.noaff_surf_mat_Mean_har_n211_top50.npy'))  

    emb = gradient.embedding.PCAMaps(n_components = comp_num) 
    print(emb)
    
# make referece
emb.fit(grp_aff)
ref_lam = emb.lambdas_ 
ref_PC = emb.maps_ 

n = len(list_aff)
print(len(list_aff))
lam, grad, vec = [None] * n, [None] * n, [None] * n
for i, x1 in enumerate(list_aff):
    print(i,' ', end = '', flush = True)
    emb.fit(x1)
    lam[i], grad[i] = emb.lambdas_ , emb.maps_

pa = gradient.ProcrustesAlignment(n_iter=10)
pa.fit(grad, reference=ref_PC)
aligned = np.array(pa.aligned_)

# np.save(f'{path_work}/noaff_surf_mat_Mean_har_n{len(file_list)}_gradients_top50',aligned)

In [ ]:
# Noise regressed out

save = False

subj_num = 103
thresh = 'top50' # top01 top03 top05 top10 top25 top30 top50 nothresh 

for i in [0,1,2]:

    pc_num = i
    data = f'noaff_surf_mat_Mean_har_n{subj_num}_gradients_{thresh}.npy'
    gradients = np.load(join(path_work,data))
    grad_num = pc_num + 1
    ind_PCs = gradients[:,:,pc_num]

    import statsmodels as sm
    from statsmodels.api import GLM

    list_reg_out = []
    for i in range(ind_PCs.shape[1]):
        print(i, ' ', end='', flush=True)
        a = GLM(ind_PCs[:,i],sm.api.add_constant(np.array([Age,FD,Age*FD]).T))

        res = a.fit()
        # res.summary()

        list_reg_out.append(res.resid_response)

    ind_PCs_reg_out = np.array(list_reg_out).T

    grp_ASD_PCs_reg_out = ind_PCs_reg_out[ASD_index,:]
    grp_TD_PCs_reg_out = ind_PCs_reg_out[TD_index,:]

    save = False
    PCs_sorted_reg_out = np.concatenate((grp_ASD_PCs_reg_out,grp_TD_PCs_reg_out), axis = 0)

    # print(pc_num)

    if save:
        print('save')
        np.save(join(path_data,f'Gradient{grad_num}_Surf_pearson_sorted_regout_gender_Mean_har_n{subj_num}_{thresh}_ABD2.npy'), PCs_sorted_reg_out)

In [ ]:
# Make Feature Selction file (ABIDE 1 Feature using)

subj_num = 103 # 103, 93
thresh = 'top50' # nothresh top10 top25 top30 top50
pc_num = 0
grad_num = pc_num + 1

file = f'Gradient{grad_num}_Surf_pearson_sorted_regout_gender_Mean_har_n{subj_num}_{thresh}_ABD2.npy'

PCs_sorted_reg_out = np.load(join(path_data,file))
label_sorted = np.concatenate((np.array(label[ASD_index]), np.array(label[TD_index])), axis = 0)
sorted_idx = np.concatenate((ASD_index,TD_index), axis = 0)

feature = np.load(join(path_data,f'G{grad_num}_{thresh}_feature_ABD1.npy'))

save_name = f'Gradient{grad_num}_Surf_pearson_sorted_regout_gender_Mean_har_n{subj_num}_{thresh}_ABD2_FeaSel_Valid.mat'

np.save(join(path_data,f'Gradient{grad_num}_Surf_pearson_sorted_regout_gender_Mean_har_n{subj_num}_{thresh}_ABD2_FeaSel_Valid.npy'), PCs_sorted_reg_out[:,feature])
sc.io.savemat(join(path_data,save_name), {'total_group_data': np.concatenate((PCs_sorted_reg_out[:,feature],label_sorted.reshape(-1,1)),axis=1)})

In [ ]:
# Run Funtaional Random Forest algorithm in matlab using save file